In [1]:
#Download all dependencies, only need to do this once
#import sys
#!{sys.executable} -m pip install spacy
#!{sys.executable} -m spacy download en_core_web_lg
#!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz
#!{sys.executable} -m pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bionlp13cg_md-0.4.0.tar.gz

import nltk
#nltk.download('stopwords')
#nltk.download('punkt')
from nltk.corpus import stopwords
from nltk import tokenize
STOPWORDS = set(stopwords.words('english'))
import csv
import spacy

import numpy as np
import pandas as pd

2021-07-19 15:27:58.887862: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-19 15:27:58.887886: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
df = pd.read_csv("positive_abstract_set.csv")
df

,disease,pmid,abstract,epi_prob,is_epi
0,"Deafness, autosomal dominant nonsyndromic sens...",33962637,<h4>Background</h4>The incidence of hydrocepha...,0.998725,True
1,Anencephaly and spina bifida X-linked,33962637,<h4>Background</h4>The incidence of hydrocepha...,0.998725,True
2,"Methylmalonic aciduria, cblB type",33990817,<h4>Introduction</h4>Limited data are availabl...,0.998415,True
3,Doyne honeycomb retinal dystrophy,33712046,Juvenile Idiopathic Arthritis is one of the mo...,0.998407,True
4,Malonyl-CoA decarboxylase deficiency,33185984,<h4>Aim</h4>To investigate the point prevalenc...,0.998325,True
...,...,...,...,...,...
750,CD4 deficiency,34128866,<h4>Abstract</h4>Previous studies have suggest...,0.519289,True
751,Muscle eye brain disease,31344241,<h4>Background and objective</h4>To understand...,0.514754,True
752,Enchondromatosis dwarfism deafness,32355641,Anticancer drug nephrotoxicity is an important...,0.512264,True
753,Severe congenital neutropenia autosomal dominant,33554218,Coronavirus disease 2019 (COVID-19) is emergin...,0.511204,True


In [3]:
abstracts = []
pmids = []

with open("positive_abstract_set.csv", newline='', encoding ='utf-8') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    for row in reader:
        abstract = row[2]
        # Remove stopwords
        for word in STOPWORDS:
            token = ' ' + word + ' '
            abstract = abstract.replace(token, ' ')
            abstract = abstract.replace(' ', ' ')
        # Only keep the article if the abstract has more than 5 chars, and it's not one of the curator articles
        if len(abstract)>5:
            abstracts.append(abstract)
            pmids.append(row[1])

In [ ]:
def remove_stopwords(string):
    for word in STOPWORDS:
        token = ' ' + word + ' '
        string = string.replace(token, ' ')
        string = string.strip()
    return string

In [ ]:
abstracts =[]
for paragraph in df['abstract']:
    paragraph = remove_stopwords(paragraph)
    abstracts.append(paragraph)

In [4]:
for i in range(7,9):
    print(pmids[i])
    print(abstracts[i])

0.9982064962387085
Neuroendocrine tumors (NETs) rare neoplasms, estimated annual incidence 6.9/100 000. They arise cells diffuse endocrine system, mainly dispersed throughout gastrointestinal (GI), pancreatic, respiratory tracts. The incidence GI-NETs recently begun show steady increase. According Surveillance, Epidemiology, End Results database, 53% patients NETs present localized disease, 20% locoregional disease, 27% distant metastases time diagnosis. Surgery mainstay treatment locoregional GI-NETs. Endoscopic resection option well-differentiated early GI-NETs, thought rarely metastasize lymph nodes. A lesion technically difficult resect via endoscopy indication local resection (partial resection without lymph node dissection). GI-NETs possible lymph node metastasis indication enterectomy lymph node dissection. For NETs metastatic lesions, cytoreduction surgery control hormonal hypersecretion alleviate symptoms; therefore, cytoreduction surgery recommended. The indications surgery v

In [ ]:
# load spaCy models
nlp = spacy.load('en_core_web_lg')

In [ ]:
# Standardize the text of the abstract by replacing all named entities with their entity label
# Eg: 3 patients were seen in a clinic in England --> CARDINAL patients were seen in a clinic in GPE
def standardizeAbstract(abstract):
    doc = nlp(abstract)
    newAbstract = abstract
    # interate through the entities in the abstract
    for e in reversed(doc.ents):
        # replace entities with their label
        #GPE = GeoPolitical Entity
        if e.label_ in {'PERCENT','CARDINAL','GPE','LOC','DATE','TIME','QUANTITY','ORDINAL'}:
            start = e.start_char
            end = start + len(e.text)
            newAbstract = newAbstract[:start] + e.label_ + newAbstract[end:]
    return newAbstract